In [1]:
!pip install pytorchvideo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 KB 977.8 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 KB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 27.1 MB/s eta 0:00:00
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.5-py3-none-any.whl size=188714 sha256=70154b57758b44faa02958e92b55ecb2ac1db3ba701ca2fd858a4311820b7465
  Stored in directory: /root/.cache/pip/wheels/e8/51/05/053b29bac2400cbbae2fb7cfc41afd280d627bca7c9363ca80
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61288 sha256=4ea6eb6802f6d41e8d9bba5d967e5171da4449eeff4977fd1f922f3c9246d50c
  Stored in directory: /root/.cache/pip/wheels/68/20/f9/a11a0dd63f4c13678b2a5ec488e48078756505c7777b75b29e
Successfully built pytorchvideo fvcore


In [2]:
start= 14
end = 21

In [3]:
import torch
import numpy as np
from torch.utils.data import (
    Dataset,
    DataLoader,
) 
import pickle


import os

import math
import torch.nn as nn
import torch.nn.functional as F

from torchvision.transforms import Compose, Lambda, Grayscale,Normalize, CenterCrop,Resize
#from torchvision.transforms._transforms_video import CenterCropVideo
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    UniformTemporalSubsample,
)

from tqdm import tqdm

from collections import OrderedDict

import torch.optim as optim



class SignDataset(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y


    def __len__(self):

        return len(self.y)

        # length = 0
        # with open(self.file, 'rb') as f:
        #     data = pickle.load(f)

        # for x in data:
        #     length += len(data[x])
        # return length


    def __getitem__(self, index):
        return self.x[index], self.y[index]
        

In [4]:
IMAGE_HEIGHT = 720
IMAGE_WIDTH = 800
IMAGE_CHANNEL = 1
NUM_FRAMES = 25
NUM_CLASSES = 60



train_inputs =[] #x
train_classes = [] #y
val_inputs = []
val_classes = []

def transform_data(x, mean, std):
    
    transform =  ApplyTransformToKey(
        key="video",
        transform=Compose(
            [

                Lambda(lambda x: x.permute(1,0,2,3)),#(frames(depth), channel, height, width) -> (channel, frames(depth), height, width)

                UniformTemporalSubsample(NUM_FRAMES),
                Lambda(lambda x: x.permute(1,0,2,3)),#(frames(depth), channel, height, width)
                Lambda(lambda x: x/255.0),
                
                Normalize((mean,), (std,)),

                #CenterCrop([720,854]),
                Resize([512,610]),
                Lambda(lambda x: x.permute(1,0,2,3)),#(channel, frames(depth), height, width)

            ]

        ),
    )
    
    return transform(x)


def load_dataloaders(batch_size,start,end):
    with open('../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_GT.pkl', 'rb') as f:
        data = pickle.load(f)


# keys are files so iterate only limited files due to memory limitations.
    for key in list(data.keys())[start:end]:
        filename = key
        print("file",filename)
        video = EncodedVideo.from_path("../input/signdataset/sign/MSSL_Train_Set/TRAIN/MSSL_TRAIN_SET_VIDEOS_ELAN/"+filename+".mp4")
    # file functions

        for x in data[key]:
            train_classes.append(x[0])
            start_time = x[1]
            end_time = x[2]
   #give path
            
    
            
            
            video_data = video.get_clip(start_sec=float(start_time)/1000.0, end_sec=float(end_time)/1000.0)
            #print(video_data["video"].shape)

            
            video_data["video"] = Grayscale(num_output_channels=1)((video_data["video"]).permute(1,0,2,3))
#             video_data["video"] = video_data["video"]/255
            #print(video_data["video"].shape)
            
            std, mean = torch.std_mean(video_data["video"])
            std = std/255.0
            mean = mean/255.0
        
            
            
            video_data = transform_data( video_data, mean, std)

        # Move the inputs to the desired device
            train_inputs.append(video_data["video"])

    signds = SignDataset(train_inputs, train_classes)
    trainlen = int(len(signds)*0.8)
    
    train_set, val_test_set = torch.utils.data.random_split(signds, [trainlen, len(signds)-trainlen])
    trainloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
    
    vallen= int(len(val_test_set)*0.8)
    val_set, test_set = torch.utils.data.random_split(val_test_set, [vallen, len(val_test_set)-vallen ])
    valloader = DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers=2)
    testloader = DataLoader(test_set, batch_size=batch_size, shuffle=True, num_workers=2)
    return trainloader, valloader, testloader 
        
    



In [5]:
class conv_3d(nn.Module):
    def __init__(self):
        super(conv_3d, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv3d(1, 64, kernel_size=5, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2)))
        
        self.conv2 = nn.Sequential(
            nn.Conv3d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2)))
        

        self._features = nn.Sequential(
            self.conv1,
            self.conv2
        )

    def forward(self, x):
        return self._features(x)# batch size, num channels,frames,  height, width

class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes, dropRate=0.0):
        super(BasicBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.droprate = dropRate
    def forward(self, x):
        out = self.conv1(self.relu(self.bn1(x)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, training=self.training)
        return torch.cat([x, out], 1)

class BottleneckBlock(nn.Module):
    def __init__(self, in_planes, out_planes, dropRate=0.0):
        super(BottleneckBlock, self).__init__()
        inter_planes = out_planes * 4
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, inter_planes, kernel_size=1, stride=1,
                               padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(inter_planes)
        self.conv2 = nn.Conv2d(inter_planes, out_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.droprate = dropRate
    def forward(self, x):
        out = self.conv1(self.relu(self.bn1(x)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, inplace=False, training=self.training)
        out = self.conv2(self.relu(self.bn2(out)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, inplace=False, training=self.training)
        return torch.cat([x, out], 1)

class TransitionBlock(nn.Module):
    def __init__(self, in_planes, out_planes, dropRate=0.0):
        super(TransitionBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=1,
                               padding=0, bias=False)
        self.droprate = dropRate
    def forward(self, x):
        out = self.conv1(self.relu(self.bn1(x)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, inplace=False, training=self.training)
        return F.avg_pool2d(out, 2)
    
    
class DenseBlock(nn.Module):
    def __init__(self, nb_layers, in_planes, growth_rate, block, dropRate=0.0):
        super(DenseBlock, self).__init__()
        self.layer = self._make_layer(block, in_planes, growth_rate, nb_layers, dropRate)
    def _make_layer(self, block, in_planes, growth_rate, nb_layers, dropRate):
        layers = []
        for i in range(nb_layers):
            layers.append(block(in_planes+i*growth_rate, growth_rate, dropRate))
        return nn.Sequential(*layers)
    def forward(self, x):
        return self.layer(x)

class DenseNet3(nn.Module):
    def __init__(self, depth=10, num_classes=60, growth_rate=12,
                 reduction=0.5, bottleneck=True, dropRate=0.4):
        super(DenseNet3, self).__init__()
        in_planes = 2 * growth_rate
        n = (depth - 4) / 3
        if bottleneck == True:
            n = n/2
            block = BottleneckBlock
        else:
            block = BasicBlock
        n = int(n)
        # 1st conv before any dense block
        self.conv1 = nn.Conv2d(128, in_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        # 1st block
        self.block1 = DenseBlock(n, in_planes, growth_rate, block, dropRate)
        in_planes = int(in_planes+n*growth_rate)
        self.trans1 = TransitionBlock(in_planes, int(math.floor(in_planes*reduction)), dropRate=dropRate)
        in_planes = int(math.floor(in_planes*reduction))
        # 2nd block
        self.block2 = DenseBlock(n, in_planes, growth_rate, block, dropRate)
        in_planes = int(in_planes+n*growth_rate)
        self.trans2 = TransitionBlock(in_planes, int(math.floor(in_planes*reduction)), dropRate=dropRate)
        in_planes = int(math.floor(in_planes*reduction))
        # 3rd block
        self.block3 = DenseBlock(n, in_planes, growth_rate, block, dropRate)
        in_planes = int(in_planes+n*growth_rate)
        # global average pooling and classifier
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu = nn.ReLU(inplace=True)
        self.fc = nn.Linear(in_planes, num_classes)
        self.in_planes = in_planes

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()
    def forward(self, x):
        out = self.conv1(x)
        
        out = self.trans1(self.block1(out))
        
        out = self.trans2(self.block2(out))
        
        out = self.block3(out)
        
        out = self.relu(self.bn1(out))
        
        out = F.avg_pool2d(out, 3)
        
        
        return out
        
        
def flatten(t):
    t = t.reshape(1, -1)
    t = t.squeeze()
    return t
  
class TimeDistributed(nn.Module):
    def __init__(self, module, batch_first=True):
        super(TimeDistributed, self).__init__()
        self.module = module
        self.batch_first = batch_first

    def forward(self, x):
        
        x = x.permute(0,1,3,4,2)# (batch, channels, height, width, frames)
        tList = [flatten(self.module(m)) for m in torch.unbind(x, dim=4) ]
        y = torch.stack(tList, dim=0)
        # We have to reshape Y
        if self.batch_first:
            y = y.contiguous().view(x.size(0), x.size(4),-1)  # (samples, timesteps, output_size)
            
        else:
            y = y.view(x.size(4), x.size(0), -1)  # (timesteps, samples, output_size)
        return y
    

               
        


class lstm(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(lstm, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)

    def forward(self, x):
        #print(inputs.shape)
        # Run through LSTM layer
        out, _ = self.lstm(x)
        out = out[:,-1,:]
        out = out.view(x.size(0),-1)
        return out
    



  

In [6]:

def test(model, device, loader, mode="Validate"):
    
    
    model.eval()

    correct = 0
    num_samples = 0
    
        
    for batch_idx, (data, target) in enumerate(loader):
        with torch.no_grad():    
            data = data.to(device)
            target = (target).to(device)

            output = model(data.half())  
            
            pred = output.max(1, keepdim=True)[1]# get the index of the max log-probability
            
            num_samples += pred.shape[0]
            
#             print("num_samples:", num_samples)
            #print(torch.sum(pred==target))
            correct += (pred == target).sum().item()
            #print("correct", correct)

    acc = 100.0 * correct / num_samples
    
    print('{} Accuracy: {}/{} ({:.0f}%)'.format(
                mode,correct, num_samples,
                100. * correct / num_samples, acc))
    
def trainonval(model, device,validloader, optimizer, criterion, epochs):

    
    model.train()
    
    print("Training on Validation Set starts")
    for epoch in range(epochs):
        correct = 0
        num_samples = 0
        train_loss = 0
        
        for batch_idx, (data, target) in enumerate(validloader):
            
            data = data.to(device)
            target = target.to(device)

            optimizer.zero_grad()
            output = model(data.half())
            
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            
            train_loss += loss.item()
            
            pred = (output.data).max(1, keepdim=True)[1]# get the index of the max log-probability
            num_samples += pred.shape[0]
            correct += torch.sum(pred.data==target)
            
        
        train_loss /= num_samples
        
        if (100 * correct / num_samples) >= 90:
            
            print('Epoch: {} , Training Accuracy on Val set: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch+1, correct, num_samples,100. * correct / num_samples, train_loss))
            test(model, device, testloader, mode = "Test after Training on validation set")
                
            return model, optimizer
            
    
        print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch+1, correct, num_samples,
                100. * correct / num_samples, train_loss))
        
    return model, optimizer

def train(model, device, train_loader,validloader,testloader, optimizer,scheduler, criterion, epochs):
    print("Train start")
    breakout = False
    
    model.train()

    
    
    for epoch in range(epochs):
        correct = 0
        num_samples = 0
        train_loss = 0
        
        for batch_idx, (data, target) in enumerate(train_loader):
            
            data = data.to(device)
            target = target.to(device)

            optimizer.zero_grad()
            output = model(data.half())
            
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            
            train_loss += loss.item()
            
            pred = output.max(1, keepdim=True)[1]# get the index of the max log-probability
            num_samples += pred.shape[0]
            correct += torch.sum(pred==target)
            
            
        scheduler.step()
        train_loss /= num_samples
        
        if (100 * correct / num_samples) >= 95:
            
            print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch+1, correct, num_samples,100. * correct / num_samples, train_loss))
            
            test(model,device,validloader, mode = "Validating")
            test(model, device, testloader, mode = "Test before Training on validation set")
            model, optimizer = trainonval(model, device, validloader, optimizer, criterion, epoch)
            test(model, device, testloader, mode = "Test after training on validation set")
            
            
            
            torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()}, "./model_optimizer.pt")
            
            breakout = True
            break
            
            
            
       
        

        print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch+1, correct, num_samples,
                100. * correct / num_samples, train_loss))
        if (((epoch+1)%5) == 0) :
            test(model,device,validloader)
            model.train()
            
    if breakout:
            print("breakout")
            return
    
    
    
    model, optimizer = trainonval(model, device, validloader, optimizer, criterion, epoch)
    test(model, device, testloader, mode= "test set ")
    
    
        
    torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()}, "./model_optimizer.pt")
    
    
        


In [7]:
n_classes = 60


model = nn.Sequential(OrderedDict([
    ('frontend', conv_3d()),
    ('features', TimeDistributed(DenseNet3())),
    ('backend', lstm(input_size=3240, hidden_size=1620)),
    ('fc', nn.Sequential( nn.Dropout(p=0.5), nn.Linear(1620, 60)))
]))





# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
# specify optimizer and learning rate
optimizer = optim.SGD(
  [
        {"params": model.fc.parameters(), "lr": 1e-2},
        {"params": model.features.parameters(), "lr": 1e-2},
        {"params": model.backend.parameters(), "lr": 1e-2},
        {"params": model.frontend.parameters(), "lr": 1e-2},
  ],
  momentum = 0.9
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
state = torch.load("../input/sign-classification/model_optimizer.pt")
model.load_state_dict(state['model_state_dict'])
model.half()
model.cuda()

optimizer.load_state_dict(state['optimizer_state_dict'])

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trainloader, valloader, testloader = load_dataloaders(batch_size=4,start=start, end=end)#73 not included 


file p05_n037
file p04_n038
file p06_n055
file p01_n099
file p02_n066
file p04_n035
file p01_n057


In [9]:
print('Number of model parameters: {}'.format(sum([p.data.nelement() for p in model.parameters()])))

Number of model parameters: 31881698


In [10]:
train(model,device,trainloader,valloader, testloader,optimizer,scheduler,criterion,35)

Train start
Epoch: 1 , Training Accuracy: 17/87 (20%) Training Loss: 1.271821
Epoch: 2 , Training Accuracy: 17/87 (20%) Training Loss: 1.225418
Epoch: 3 , Training Accuracy: 25/87 (29%) Training Loss: 1.177712
Epoch: 4 , Training Accuracy: 22/87 (25%) Training Loss: 1.150615
Epoch: 5 , Training Accuracy: 25/87 (29%) Training Loss: 1.151917
Validate Accuracy: 3/17 (18%)
Epoch: 6 , Training Accuracy: 24/87 (28%) Training Loss: 1.122328
Epoch: 7 , Training Accuracy: 28/87 (32%) Training Loss: 1.146237
Epoch: 8 , Training Accuracy: 26/87 (30%) Training Loss: 1.164691
Epoch: 9 , Training Accuracy: 25/87 (29%) Training Loss: 1.117030
Epoch: 10 , Training Accuracy: 34/87 (39%) Training Loss: 1.080729
Validate Accuracy: 4/17 (24%)
Epoch: 11 , Training Accuracy: 33/87 (38%) Training Loss: 1.042632
Epoch: 12 , Training Accuracy: 33/87 (38%) Training Loss: 0.992569
Epoch: 13 , Training Accuracy: 35/87 (40%) Training Loss: 1.059492
Epoch: 14 , Training Accuracy: 25/87 (29%) Training Loss: 1.044832

In [11]:
 #test(model, device, testloader)

In [12]:
#from 21 15 epochs